In [1]:
!pip install razdel
!pip install pymorphy2
!pip install pyLDAvis
!pip install langdetect
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score
import re
from collections import Counter

import razdel
import nltk
nltk.download('stopwords')
import pymorphy2
import gensim
import numpy as np
plt.style.use('ggplot')
import plotly
from tqdm import tqdm_notebook
from sklearn.manifold import TSNE

from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
import pyLDAvis
import pyLDAvis.gensim

     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 8.2MB 3.5MB/s 
     |████████████████████████████████| 1.6MB 5.5MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=26a3ca3005f31b01108aa85c7d196ee9e26c22b7b0701524d3871991fcbb0dd1
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis
     |████████████████████████████████| 983kB 4.2MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993195 sha256=b1209db2891b1e978f5f78ba07984ea91fbd9259b1988054abd94978acb7bfcc
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/data7/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/data7/test.csv')

In [5]:
#train_df = pd.read_csv('train.csv')
#test_df = pd.read_csv('test.csv')

## Выбросим всё лишнее

In [6]:
def to_lower(text):
    return text.str.lower()

In [7]:
not_len = [56, 60, 63, 49, 66, 40]
not_str = ['що це ми не як на вони та але ви до про він оплески коли', 
           'the and of it that you to apos we this is in they so are',
          'гэта што калі мы не яны як але ён каб дзякуй больш пра на вы',
          'бұл мен біз бір үшін ол деп және емес бар керек бірақ олар қол осы',
          'що це тому ми чи дуже які щоб дякую якщо було вона від мене ось',
          'это что мы не как на вы они но из то он так для аплодисменты',
          '笑声 掌声 谢谢 现在 所以 事实上 当然 鼓掌 但是 因此 那么 非常感谢 谢谢大家 是的 好吧',
          '笑聲 掌聲 謝謝 所以 現在 事實上 當然 因此 謝謝大家 對吧 但是 鼓掌 謝謝各位 他說 我說']
train_df = train_df.loc[~train_df.sentence.str.len().isin(not_len)].apply(to_lower)
train_df = train_df.loc[~train_df.sentence.isin(not_str)].apply(to_lower)

## Построим список стоп слов

In [8]:
train_df['k'] = ' '
train_df['gg'] = train_df[['language', 'k', 'sentence']].sum(axis = 1)

In [9]:
tt = {}
for i in tqdm_notebook(train_df['gg']):
    h = i.split()
    for j in h:
        if h[0] in tt:
            tt[h[0]].add(j) 
        else:
            tt[h[0]] = set()
            tt[h[0]].add(j)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



In [10]:
Spisok_stop = tt['fr'].copy()
for i in tt.items():
    #print(type(i[1]))
    Spisok_stop.intersection_update(i[1])

In [11]:
Spisok_stop = Spisok_stop.union({'!', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '@', '#', '$', '<', '>', '+'})

## Обучаем и предсказываем

In [12]:
x = train_df.values[:,  0]
label_encoder = LabelEncoder().fit(train_df.values[:, 1])
y = label_encoder.transform(train_df.values[:, 1])

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [13]:
from sklearn.feature_extraction.text import  CountVectorizer
pipe = Pipeline([
    ('vectorizer', CountVectorizer(
            stop_words=Spisok_stop, min_df=3, max_df=0.78, token_pattern=r"[^-\s\d.,;!]+"
                    )
    ),
    ('model', SGDClassifier(random_state=42, loss='log', class_weight='balanced')),
])

In [14]:
pipe.fit(x, y)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['&apos', '&quot'] not in stop_words.



Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.78,
                                 max_features=None, min_df=3,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words={'!', '#', '$', '%', '&apos;',
                                             '&quot;', '(', ')', '+', ',', '-',
                                             '.', '...', '0', '1', '2', '3'...
                 SGDClassifier(alpha=0.0001, average=False,
                               class_weight='balanced', early_stopping=False,
                               epsilon=0.1, eta0=0.0, fit_intercept=True,
                               l1_ratio=0.15, learning_rate='optimal',
                 

In [15]:
submit_predictions = pipe.predict(test_df.sentence.values)

In [16]:
test_df['language'] = label_encoder.classes_[submit_predictions]


In [17]:
test_df[['index',  'language']].to_csv('from_Russia_with_love_end.csv', index=False)